In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [ ]:
def merge_csvs_keep_header_once(csv1_path, csv2_path, output_path):
    """
    Merge two CSV files into one, keeping only the first header row.

    Parameters:
    - csv1_path (str): Path to the first CSV file
    - csv2_path (str): Path to the second CSV file
    - output_path (str): Path where the merged CSV will be saved
    """
    df1 = pd.read_csv(csv1_path)
    df2 = pd.read_csv(csv2_path)

    # Concatenate the dataframes
    merged_df = pd.concat([df1, df2], ignore_index=True)

    # Save to output
    merged_df.to_csv(output_path, index=False)
    print(f"Merged CSV saved to: {output_path}")

In [ ]:
merge_csvs_keep_header_once(
    "summary_tables/antibodies/scaffold_scores_antibodies_dbg.csv",
    "summary_tables/policlonals/scaffold_scores_policlonals_dbg.csv",
    "scaffold_scores_for_radar_plot_dbg.csv",
)

In [ ]:
def generate_combined_radar_plots_show_and_save(csv_path, ass_method):
    df = pd.read_csv(csv_path)

    metrics = [
        "total_sequences_scaled",
        "max_length_scaled",
        "N50_scaled",
        "coverage_scaled",
        "composite_score",
    ]
    clean_labels = [label.replace("_", " ") for label in metrics]
    clean_labels_closed = clean_labels + [clean_labels[0]]

    color_antibodies = "#1b9e77"
    color_policlonals = "#e6ab02"
    radar_bg_color = "#f5f5f5"
    overall_bg_color = "white"

    indices = ["1", "2", "3"]

    for idx in indices:
        ma_light = f"ma{idx}light"
        pama_light = f"pama{idx}light"
        ma_heavy = f"ma{idx}heavy"
        pama_heavy = f"pama{idx}heavy"

        if not all(
            r in df["run"].values for r in [ma_light, pama_light, ma_heavy, pama_heavy]
        ):
            print(f"Skipping index {idx} due to missing runs.")
            continue

        row_ma_light = df[df["run"] == ma_light].iloc[0][metrics]
        row_pama_light = df[df["run"] == pama_light].iloc[0][metrics]
        row_ma_heavy = df[df["run"] == ma_heavy].iloc[0][metrics]
        row_pama_heavy = df[df["run"] == pama_heavy].iloc[0][metrics]

        light_ma_values = row_ma_light.tolist() + [row_ma_light[metrics[0]]]
        light_pama_values = row_pama_light.tolist() + [row_pama_light[metrics[0]]]
        heavy_ma_values = row_ma_heavy.tolist() + [row_ma_heavy[metrics[0]]]
        heavy_pama_values = row_pama_heavy.tolist() + [row_pama_heavy[metrics[0]]]

        fig = make_subplots(
            rows=1,
            cols=2,
            specs=[[{"type": "polar"}, {"type": "polar"}]],
            subplot_titles=(
                f"{ma_light} vs {pama_light}",
                f"{ma_heavy} vs {pama_heavy}",
            ),
            horizontal_spacing=0.25,
        )

        # LIGHT
        fig.add_trace(
            go.Scatterpolar(
                r=light_ma_values,
                theta=clean_labels_closed,
                fill="toself",
                name="antibodies",
                line=dict(color=color_antibodies),
                showlegend=True,
            ),
            row=1,
            col=1,
        )

        fig.add_trace(
            go.Scatterpolar(
                r=light_pama_values,
                theta=clean_labels_closed,
                fill="toself",
                name="policlonals",
                line=dict(color=color_policlonals),
                showlegend=True,
            ),
            row=1,
            col=1,
        )

        # HEAVY
        fig.add_trace(
            go.Scatterpolar(
                r=heavy_ma_values,
                theta=clean_labels_closed,
                fill="toself",
                name="antibodies",
                line=dict(color=color_antibodies),
                showlegend=False,
            ),
            row=1,
            col=2,
        )

        fig.add_trace(
            go.Scatterpolar(
                r=heavy_pama_values,
                theta=clean_labels_closed,
                fill="toself",
                name="policlonals",
                line=dict(color=color_policlonals),
                showlegend=False,
            ),
            row=1,
            col=2,
        )

        fig.update_layout(
            height=600,
            width=1100,
            title_text=f"Radar Plot – ma{idx} vs pama{idx} {ass_method}",
            showlegend=True,
            polar=dict(
                bgcolor=radar_bg_color,
                radialaxis=dict(visible=True, range=[0, 1], gridcolor="white"),
                angularaxis=dict(gridcolor="white"),
            ),
            polar2=dict(
                bgcolor=radar_bg_color,
                radialaxis=dict(visible=True, range=[0, 1], gridcolor="white"),
                angularaxis=dict(gridcolor="white"),
            ),
            paper_bgcolor=overall_bg_color,
            plot_bgcolor=overall_bg_color,
        )

        fig.show()
        filename = f"radar_ma{idx}_pama{idx}_{ass_method}.svg"
        fig.write_image(filename)
        print(f"Saved: {filename}")

In [ ]:
generate_combined_radar_plots_show_and_save(
    "scaffold_scores_for_radar_plot_dbg.csv", "dbg"
)